<a href="https://colab.research.google.com/github/trevorbehnke/Google-Cloud-AI-Platform-XGBoost-BigQuery/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure XGBoost is installed




In [1]:
!pip3 install xgboost

Import Packages

In [0]:
import pandas as pd
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery

Add Google Cloud project

In [3]:
!gcloud config set project mlxgboost

Updated property [core/project].


Authenticate Account

In [4]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=6D2h88X8S80EQ_RNKFjtZ_E4QiXUAWyCtkPXkU5crAs&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/xQG36HLfLk8rsujliFNxtSr8OyNSdTaY3Tjuj3yGDeA8zNmDlxeUi6o

You are now logged in as [me@trevorbehnke.com].
Your current project is [mlxgboost].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Create Service Account

In [0]:
!gcloud iam service-accounts create mlxgboost

Add IAM Policy to Service Account

In [6]:
!gcloud projects add-iam-policy-binding mlxgboost --member "serviceAccount:mlxgboost@mlxgboost.iam.gserviceaccount.com" --role "roles/owner"

Updated IAM policy for project [mlxgboost].
bindings:
- members:
  - serviceAccount:service-454613473835@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:454613473835-compute@developer.gserviceaccount.com
  - serviceAccount:454613473835@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - serviceAccount:service-454613473835@cloud-ml.google.com.iam.gserviceaccount.com
  role: roles/ml.serviceAgent
- members:
  - serviceAccount:mlxgboost@mlxgboost.iam.gserviceaccount.com
  - user:me@trevorbehnke.com
  role: roles/owner
etag: BwWgW3HegoE=
version: 1


Create Service Account Credentials

In [7]:
!gcloud iam service-accounts keys create mlxgboost.json --iam-account mlxgboost@mlxgboost.iam.gserviceaccount.com

created key [b9588a7ef583357bda228ccec57c1a113aae8488] of type [json] as [mlxgboost.json] for [mlxgboost@mlxgboost.iam.gserviceaccount.com]


Create Environment Variables for Service Account Credentials

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/mlxgboost.json"

Save Service Account Credentials to Local Folder

In [0]:
!export GOOGLE_APPLICATION_CREDENTIALS="/content/mlxgboost.json"

Ingest Data using BigQuery

In [10]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


Brief Statistical Description of Data

In [11]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


Example of Value Counts for Non-Numerical Data

In [12]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

Drop Rows With Null Values and Shuffle the Data

In [0]:
df = df.dropna()
df = shuffle(df, random_state=2)

Extract the Label Column Into a Separate Variable and Create a DataFrame With Only Our Features

In [0]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])

Preview the New Dataset

In [15]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,True,32,1,41.0
6132,False,28,1,30.0
5986,False,44,1,38.0
7682,False,34,1,38.0
4910,True,31,1,40.0


Convert Categorical Features into Integers

In [0]:
data['is_male'] = data['is_male'].astype(int)

Split Data Into Train and Test Sets Using Scikit Learn *train_test_split*

In [0]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

Create the XGBoost Model

In [0]:
model = xgb.XGBRegressor(
    objective='reg:linear'
)

Train the Model

In [19]:
model.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[17:45:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

Generate Predictions On the Test Data

In [0]:
y_pred = model.predict(x_test)

View Model Performance on the Test Set

In [23]:
for i in range(5):
    print('Predicted weight: ', y_pred[i])
    print('Actual weight: ', y_test.iloc[i])
    print()

Predicted weight:  7.859754
Actual weight:  8.2783579381

Predicted weight:  7.4478636
Actual weight:  8.313631900019999

Predicted weight:  7.672204
Actual weight:  7.1870697412

Predicted weight:  8.020563
Actual weight:  8.8625829324

Predicted weight:  7.0871387
Actual weight:  6.03625673356



Save Model to Local Folder

In [0]:
model.save_model('model.bst')

Define Environment Variables for the Google Cloud Storage Bucket and Model

In [0]:
GCP_PROJECT = 'mlxgboost'
MODEL_BUCKET = 'gs://mlxgboost'
VERSION_NAME = 'v1'
MODEL_NAME = 'baby_weight'

Create a Google Cloud Storage Bucket for the Model

In [26]:
!gsutil mb $MODEL_BUCKET

Creating gs://mlxgboost/...


Copy the Model to the Google Cloud Storage Bucket

In [27]:
!gsutil cp ./model.bst $MODEL_BUCKET

Copying file://./model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 63.5 KiB/ 63.5 KiB]                                                
Operation completed over 1 objects/63.5 KiB.                                     


Create the Model on Google Cloud AI Platform

In [28]:
!gcloud ai-platform models create $MODEL_NAME

Created ml engine model [projects/mlxgboost/models/baby_weight].


Deploy the Model

In [0]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--framework='XGBOOST' \
--runtime-version=1.15 \
--origin=$MODEL_BUCKET \
--python-version=3.7 \
--project=$GCP_PROJECT

Test the Deployed Model by Inputing Some Data

In [30]:
%%writefile predictions.json
[0.0, 33.0, 1.0, 27.0]
[1.0, 26.0, 1.0, 40.0]

Writing predictions.json


Print Out Your Model Predictions!

In [31]:
prediction = !gcloud ai-platform predict --model=$MODEL_NAME --json-instances=predictions.json --version=$VERSION_NAME
print(prediction.s)

[3.2076520919799805, 7.901758670806885]
